In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import EarlyStopping

import cv2
import os
from imutils import paths
import imutils
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
import models
from models import SimplePreprocessor, AnimalsDatasetManager



In [3]:
def build_cnn():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3,3), padding='same', activation= 'relu', input_shape=(32,32,3)))
    model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(layers.AveragePooling2D(pool_size=(2, 2), padding='same'))
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.AveragePooling2D(pool_size=(2, 2), padding='same'))
    model.add(layers.Flatten())
    model.add(layers.Dense(10, activation='softmax'))
    return model

In [4]:
def create_label_folder_dict(adir):
    sub_folders= [folder for folder in os.listdir(adir)
                  if os.path.isdir(os.path.join(adir, folder))]
    label_folder_dict= dict()
    for folder in sub_folders:
        item= {folder: os.path.abspath(os.path.join(adir, folder))}
        label_folder_dict.update(item)
    return label_folder_dict

In [5]:
label_folder_dict= create_label_folder_dict("../../datasets/Animals/")

In [6]:
sp = SimplePreprocessor(width=32, height=32)
data_manager = AnimalsDatasetManager([sp])
data_manager.load(label_folder_dict, verbose=100)
data_manager.process_data_label()
data_manager.train_valid_test_split()

butterfiles 500
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
cats 501
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
chickens 500
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
cows 500
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
dogs 501
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
elephants 500
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
fishes 500
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
horses 500
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
lions 500
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Processed 500/500
spiders 500
Processed 100/500
Processed 200/500
Processed 300/500
Processed 400/500
Proc

In [7]:
batch_size = 32
img_height = 32
img_width = 32

# def configure_for_performance(ds):
#     ds = ds.shuffle(buffer_size=1000)   # shuffle the dataset
#     ds = ds.batch(batch_size)    # partition the dataset to obtain a batch dataset
#     ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)  # prefetch some batches for faster processing
#     return ds

# train_ds = configure_for_performance(train_ds)
# val_ds = configure_for_performance(val_ds)

In [8]:


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, models, layers, regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from functools import partial
from models import DefaultModel


# Load pretrained model

In [9]:
from tensorflow.keras.applications import VGG19, VGG16, NASNetLarge, Xception, InceptionV3
from tensorflow.keras.applications import DenseNet121, DenseNet201

input_shape = (32,32,3)

print("Loading network...")
# base_model = VGG19(weights="imagenet", include_top=False, input_tensor=tf.keras.layers.Input(shape=input_shape))
base_model = DenseNet201(weights="imagenet", include_top=False, input_tensor=tf.keras.layers.Input(shape=input_shape))

print("Showing layers...")
# loop over the layers in the network and display them to the console
for (i, layer) in enumerate(base_model.layers):
    print("{}\t{}".format(i, layer.__class__.__name__))
    
    
    

Loading network...
74850304/74836368 [==============================] - 7s 0us/step
Showing layers...
0	InputLayer
1	ZeroPadding2D
2	Conv2D
3	BatchNormalization
4	Activation
5	ZeroPadding2D
6	MaxPooling2D
7	BatchNormalization
8	Activation
9	Conv2D
10	BatchNormalization
11	Activation
12	Conv2D
13	Concatenate
14	BatchNormalization
15	Activation
16	Conv2D
17	BatchNormalization
18	Activation
19	Conv2D
20	Concatenate
21	BatchNormalization
22	Activation
23	Conv2D
24	BatchNormalization
25	Activation
26	Conv2D
27	Concatenate
28	BatchNormalization
29	Activation
30	Conv2D
31	BatchNormalization
32	Activation
33	Conv2D
34	Concatenate
35	BatchNormalization
36	Activation
37	Conv2D
38	BatchNormalization
39	Activation
40	Conv2D
41	Concatenate
42	BatchNormalization
43	Activation
44	Conv2D
45	BatchNormalization
46	Activation
47	Conv2D
48	Concatenate
49	BatchNormalization
50	Activation
51	Conv2D
52	AveragePooling2D
53	BatchNormalization
54	Activation
55	Conv2D
56	BatchNormalization
57	Activation
58	Conv2

In [10]:

class FCHeadNet:
    @staticmethod
    def build(base_model, n_classes=5, D=100): # initialize the head model that will be placed on top of the base, then add a FC layer
        
        base_model.trainable= False # only allow new classifer on top of head to be update
        head_model = base_model.output
        head_model = tf.keras.layers.Flatten(name="flatten")(head_model)
        head_model = tf.keras.layers.Dense(D, activation="relu")(head_model)
        head_model = tf.keras.layers.Dropout(0.3)(head_model)
        # add a softmax layer
        head_model = tf.keras.layers.Dense(n_classes, activation="softmax")(head_model)
        return head_model

In [11]:
head_model = FCHeadNet.build(base_model, len(data_manager.classes), 256)
model = tf.keras.models.Model(inputs=base_model.input, outputs=head_model)



In [12]:
opt = tf.keras.optimizers.RMSprop(lr=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

c:\Users\PC\anaconda3\envs\tf2_cpu\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


# data augmentation & adversarial sample generation


# Training model

### We train the new model on our dataset. Note that at the moment, we only update the classifier on the top. Owning the existed patterns of `pretrained models`, we can obtain a good performance.

In [14]:
# opt = tf.keras.optimizers.Adam(learning_rate= 0.001)
# model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# early_checkpoint = EarlyStopping(patience=5, monitor='val_accuracy', mode='max', restore_best_weights=True) # same as monitoring min for val_loss
# callbacks = [early_checkpoint]

# history = model.fit(x = data_manager.X_train, y = data_manager.y_train, 
#                     validation_data = (data_manager.X_valid, data_manager.y_valid), 
#                     callbacks=callbacks,
#                     epochs = 30, batch_size = 64)

model.fit(  x = data_manager.X_train, y = data_manager.y_train,
            validation_data = (data_manager.X_valid, data_manager.y_valid), 
            epochs=5, batch_size = 64)



Epoch 1/5
63/63 [==============================] - 13s 111ms/step - loss: 1.5798 - accuracy: 0.4590 - val_loss: 1.2615 - val_accuracy: 0.5800
Epoch 2/5
63/63 [==============================] - 5s 76ms/step - loss: 1.1122 - accuracy: 0.6120 - val_loss: 1.1742 - val_accuracy: 0.6020
Epoch 3/5
63/63 [==============================] - 6s 93ms/step - loss: 0.9229 - accuracy: 0.6770 - val_loss: 1.2172 - val_accuracy: 0.5720
Epoch 4/5
63/63 [==============================] - 5s 77ms/step - loss: 0.7760 - accuracy: 0.7222 - val_loss: 1.1762 - val_accuracy: 0.6020
Epoch 5/5
63/63 [==============================] - 5s 75ms/step - loss: 0.6530 - accuracy: 0.7682 - val_loss: 1.2156 - val_accuracy: 0.5860


### We now train both the feature extractor and classifier with a small learning rate.

In [15]:
base_model.trainable = True
# model.fit(train_ds, epochs=2, validation_data=val_ds)
model.fit(  x = data_manager.X_train, y = data_manager.y_train, 
            validation_data = (data_manager.X_valid, data_manager.y_valid), 
            epochs=5, 
            batch_size = 64)

Epoch 1/5
63/63 [==============================] - 5s 73ms/step - loss: 0.5555 - accuracy: 0.8100 - val_loss: 1.3745 - val_accuracy: 0.5560
Epoch 2/5
63/63 [==============================] - 5s 74ms/step - loss: 0.4846 - accuracy: 0.8338 - val_loss: 1.2276 - val_accuracy: 0.6000
Epoch 3/5
63/63 [==============================] - 5s 75ms/step - loss: 0.4066 - accuracy: 0.8620 - val_loss: 1.3026 - val_accuracy: 0.5700
Epoch 4/5
63/63 [==============================] - 5s 74ms/step - loss: 0.3382 - accuracy: 0.8923 - val_loss: 1.3858 - val_accuracy: 0.5920
Epoch 5/5
63/63 [==============================] - 5s 73ms/step - loss: 0.2942 - accuracy: 0.9087 - val_loss: 1.3423 - val_accuracy: 0.5920


In [16]:
model.save("./models/first_model", save_format="tf")

INFO:tensorflow:Assets written to: ./models/first_model\assets


# Load Test Set

In [17]:
categorical_labels = ['butterfly', 'cat', 'chicken', 'cow', 'dog', 'elephant', 'fish', 'horse', 'lion', 'spider']

In [18]:
def load_test_set(folder):
    from PIL import Image
    from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
    image_paths = sorted(list(paths.list_images(folder)))
    test_data = []
    for i, image_path in enumerate(image_paths):
        img = Image.open(image_path).convert("RGB") #load_img(image_path)
        img = img.resize((32, 32), Image.ANTIALIAS)
        x = img_to_array(img)
        test_data.append(x)
        if i+1 % 500 ==0:
            print("Loaded {} images".format(i+1))
    test_data = np.asarray(test_data)
    return test_data

In [19]:
test_data = load_test_set("./official_test_aug/")

C:\Users\PC\AppData\Local\Temp\ipykernel_24272\1344258102.py:8: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((32, 32), Image.ANTIALIAS)


In [20]:
print("Test shape: {}".format(test_data.shape))

Test shape: (23098, 32, 32, 3)


In [21]:
preds = model.predict(test_data, batch_size= 32)

print(preds.shape)
print(preds[0:15])

(23098, 10)
[[1.00000000e+00 1.66222293e-30 5.41215121e-33 4.13263837e-35
  8.60993879e-35 2.94132744e-38 5.48004674e-17 2.33972586e-28
  0.00000000e+00 4.67001294e-20]
 [9.99999046e-01 2.45534235e-20 1.27415430e-21 1.21619964e-23
  7.54090206e-17 4.90397195e-29 4.81072170e-12 2.13190695e-25
  4.03786848e-22 9.01316128e-07]
 [1.00000000e+00 2.94228427e-21 5.73001987e-13 8.77652691e-23
  1.35024512e-21 4.57022539e-21 3.02355890e-10 2.76973311e-19
  5.36863973e-26 2.55657007e-09]
 [1.00000000e+00 6.18652581e-20 2.20789524e-19 1.30809344e-21
  2.14517362e-10 8.90570878e-35 2.38702301e-18 6.48475931e-20
  2.59480081e-31 3.10350856e-09]
 [1.00000000e+00 3.15104147e-20 1.11078070e-10 3.93101740e-19
  1.26002223e-10 6.76459851e-34 4.36367175e-12 1.66472216e-16
  1.13897328e-22 4.18416413e-11]
 [1.00000000e+00 6.80883835e-28 6.21935689e-24 6.71497143e-21
  9.09105905e-21 1.25142038e-34 2.24748228e-15 1.09121943e-22
  9.44161471e-33 1.13038821e-10]
 [1.00000000e+00 4.41864664e-20 5.07124829e-18

In [22]:
num_predicted_labels = np.argmax(preds, axis=1)
# num_predicted_labels = yourmodel.predict(test_data)

print(num_predicted_labels[0:15])

for elem in num_predicted_labels:
    print(elem)


[0 0 0 0 0 0 0 0 0 0 9 0 0 0 0]
0
0
0
0
0
0
0
0
0
0
9
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
9
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
9
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
9
0
0
0
0
0
0
0
0
0
0
9
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
0
0
0
0
0
0
0
0
9
0
0
0
0
0
0
0
0
0
6
0
0
0
0
0
0
6
0
0
0
0
0
0
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
0
0
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
0
0
0
0
0
0
0
0
0
0
0


In [23]:
cat_predicted_labels= [categorical_labels[num_predicted_labels[i]] for i in range(len(num_predicted_labels))]
print(cat_predicted_labels[0:15])



['butterfly', 'butterfly', 'butterfly', 'butterfly', 'butterfly', 'butterfly', 'butterfly', 'butterfly', 'butterfly', 'butterfly', 'spider', 'butterfly', 'butterfly', 'butterfly', 'butterfly']


In [24]:
def create_solution():
    import csv
    header= ['ID', 'Label']
    with open('my_solution.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        # write the header
        writer.writerow(header)
        data = []
        for i in range(len(cat_predicted_labels)):
            data.append([str(i), cat_predicted_labels[i]])
        writer.writerows(data)

In [25]:
create_solution()

You now can submit the csv file `my_solution.csv` to the Kaggle competition.